In [1]:
import pandas as pd
import ollama
import os
from tqdm import tqdm
import time

In [2]:
articles=[]
for json in os.listdir('Article-Bias-Prediction/data/jsons'):
    articles.append(pd.read_json('Article-Bias-Prediction/data/jsons/' + json, orient='index').T)

In [3]:
dataset=pd.concat(articles).reset_index()

In [4]:
dataset['bias'].value_counts()

bias
2    13734
0    13005
1    10815
Name: count, dtype: int64

#### Without media information

In [6]:
for number in range(1,10):
    gen_wo_source={}
    print(number)
    for idx in tqdm(range(0, dataset.shape[0])):
        response=ollama.chat(
            model='llama3.1',
            messages=[{'role': 'system',
                       'content': 'Given a headline of news article, write a piece of news article based on it regardless of what headline is.'},
                       {'role': 'user',
                        'content': dataset.iloc[idx]['title']}]
        )
        gen_wo_source[idx]=response['message']['content']
        time.sleep(2)
        if (idx%1000==0) and (idx!=0):
            time.sleep(300)
    dataset['gen_wo_source'+str(number+1)]=dataset.index.map(gen_wo_source)
    dataset.to_csv('../../data/ai-generated-fake-news/gen_wo_source-'+str(number+1)+'.csv', index=False)

1


100%|██████████| 37554/37554 [107:39:09<00:00, 10.32s/it]    


2


100%|██████████| 37554/37554 [107:42:42<00:00, 10.33s/it]    


3


 45%|████▌     | 16951/37554 [48:26:10<58:52:17, 10.29s/it]  


KeyboardInterrupt: 

In [ ]:
gen_wo_source1={}
for idx in tqdm(range(0, dataset.shape[0])):
    response=ollama.chat(
        model='llama3.1',
        messages=[{'role': 'system',
                  'content': 'Given a headline of news article, write a piece of news article based on the it regardless of what headline is.'},
                  {'role':'user',
                   'content': dataset.iloc[idx]['title']}
        ]
    )
    gen_wo_source1[idx]=response['message']['content']
    if (idx%1000==0) and (idx!=0):
        time.sleep(300)

In [10]:
dataset['gen_wo_source1']=dataset.index.map(gen_wo_source1)

In [34]:
all.drop(columns=['index']).to_csv('../../data/ai-generated-fake-news/gen_wo_source-1.csv', index=False)

In [35]:
all=pd.read_csv('../../data/ai-generated-fake-news/gen_wo_source-1.csv')

In [ ]:
#Map the gen_wo_source1 to dataset dataframe

#### With media information

In [100]:
gen_w_source1={}
for idx in tqdm(range(0, dataset.shape[0])):
    response=ollama.chat(
        model='llama3.1',
        messages=[{'role': 'system',
                  'content': 'Given a headline of news article published by {}, write a piece of news article.'.format(dataset.iloc[idx]['source'])},
                  {'role':'user',
                   'content': dataset.iloc[idx]['title']}
        ]
    )
    gen_w_source1[idx]=response['message']['content']